In [ ]:
#!nvidia-smi

In [ ]:
#%env CUDA_DEVICE_ORDER=PCI_BUS_ID
#%env CUDA_VISIBLE_DEVICES=0

In [1]:
import tensorflow as tf
import random
import matplotlib.pyplot as plt

from tensorflow.examples.tutorials.mnist import input_data

tf.set_random_seed(777)  # reproducibility

In [2]:
mnist = input_data.read_data_sets("../data/", one_hot=True)
# Check out https://www.tensorflow.org/get_started/mnist/beginners for
# more information about the mnist dataset

Extracting ../data/train-images-idx3-ubyte.gz
Extracting ../data/train-labels-idx1-ubyte.gz
Extracting ../data/t10k-images-idx3-ubyte.gz
Extracting ../data/t10k-labels-idx1-ubyte.gz


In [3]:
learning_rate = 0.001
training_epochs = 15
num_classes = 10
batch_size = 100

In [4]:
element_size = 28
time_steps = 28
hidden_layer_size = 128

In [5]:
X = tf.placeholder(tf.float32, shape=[None, time_steps, element_size], name='X')
Y = tf.placeholder(tf.float32, shape=[None, num_classes], name='Y')

In [6]:
rnn_cell = tf.contrib.rnn.BasicRNNCell(hidden_layer_size)
#rnn_cell = tf.contrib.rnn.BasicLSTMCell(hidden_layer_size)
#rnn_cell = tf.contrib.rnn.GRUCell(hidden_layer_size)
outputs, _ = tf.nn.dynamic_rnn(rnn_cell, X, dtype=tf.float32)

In [7]:
W = tf.Variable(tf.truncated_normal([hidden_layer_size, num_classes], stddev=0.01))
b = tf.Variable(tf.truncated_normal([num_classes], stddev=0.01))

In [8]:
last_rnn_output = outputs[:,-1,:]
logits = tf.matmul(last_rnn_output, W) + b

In [9]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [10]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [11]:
# initialize
sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth =True)))
sess.run(tf.global_variables_initializer())

In [12]:
# train my model
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0.
    avg_train_acc = 0.
    avg_test_acc = 0.
    total_batch = int(mnist.train.num_examples / batch_size)
    total_batch_test = int(mnist.test.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, time_steps, element_size)
        feed_dict = {X: batch_xs, Y: batch_ys}
        acc, c, _ = sess.run([accuracy, cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
        avg_train_acc += acc / total_batch
        
    for i in range(total_batch_test):
        batch_xs, batch_ys = mnist.test.next_batch(batch_size)        
        batch_xs = batch_xs.reshape(-1, time_steps, element_size)
        feed_dict = {X: batch_xs, Y: batch_ys}
        acc = sess.run(accuracy, feed_dict=feed_dict)
        avg_test_acc += acc / total_batch_test

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost), 
          'train accuracy = ', '{:.5f}'.format(avg_train_acc), 
          'test accuracy = ', '{:.5f}'.format(avg_test_acc))


print('Learning Finished!')

Learning started. It takes sometime.
Epoch: 0001 cost = 0.707304945 train accuracy =  0.77402 test accuracy =  0.90820
Epoch: 0002 cost = 0.276071512 train accuracy =  0.91940 test accuracy =  0.93430
Epoch: 0003 cost = 0.212505489 train accuracy =  0.93858 test accuracy =  0.95000
Epoch: 0004 cost = 0.172286048 train accuracy =  0.95165 test accuracy =  0.95540
Epoch: 0005 cost = 0.155612229 train accuracy =  0.95602 test accuracy =  0.95730
Epoch: 0006 cost = 0.139425008 train accuracy =  0.96093 test accuracy =  0.96280
Epoch: 0007 cost = 0.129894637 train accuracy =  0.96282 test accuracy =  0.96070
Epoch: 0008 cost = 0.122527156 train accuracy =  0.96495 test accuracy =  0.96630
Epoch: 0009 cost = 0.113743080 train accuracy =  0.96715 test accuracy =  0.96480
Epoch: 0010 cost = 0.106142216 train accuracy =  0.97011 test accuracy =  0.96650
Epoch: 0011 cost = 0.097220649 train accuracy =  0.97193 test accuracy =  0.96850
Epoch: 0012 cost = 0.092684838 train accuracy =  0.97349 test